Inga Ulusoy, Computational modelling in python, SoSe2020 

# Autocorrelation functions and Fourier transforms

The autocorrelation function is defined as the overlap of the propagated wave function with the wave function at $t=0$:
\begin{align}
C(t) = \langle \Psi (0) | \Psi (t) \rangle
\end{align}

The Fourier transform gives the spectrum. For an overview of available functions, see https://docs.scipy.org/doc/numpy/reference/routines.fft.html.

In [ ]:
from math import *
from numpy import *
from scipy import linalg
from scipy.constants import eV, c, h, hbar, m_e
from scipy.constants import physical_constants
import matplotlib.pyplot as plt
from scipy.integrate import ode
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
amu = physical_constants['atomic mass constant'][0]
bohr = physical_constants['Bohr radius'][0]
cm2Eh = 4.556335E-6
JtoEh = 4.359744E-18
au2fs = 0.0241888

Let's calculate the autocorrelation function for our HO example.

In [ ]:
#parameters of the HO: no of grid points for x
nsteps=500
xmin = 0
xmax = 5
#vibrational frequency in cm^-1
w = 2990
#interatomic distance in a.u.
R0 = 2.9
omega = w*cm2Eh

def get_mu(m1,m2):
    mu = m1*m2/(m1+m2)
    return mu

def get_k(mu,omega):
    k = mu*(2*pi*omega*100*c)**2
    #print(omega*100*c*10**-12,'THz')
    return k

m1 = 1*amu
m2 = 35*amu
mu = get_mu(m1,m2)
#convert this to atomic units by dividing througe m_e
muau=mu/m_e
k = get_k(mu,w)
print('Reduced mass is {:3.2e} kg ({:4.2f} au) and the force constant {:4.2f} N/m.'.format(mu,muau,k))

#set up the nuclear grid
#step size h for the numerical derivative
xgrid,h=linspace(xmin,xmax,nsteps,retstep=True)

def potential(x,omega):
    pot=(0.5*omega**2*x**2)
    return pot

# create the Hamiltonian
Hamiltonian=zeros((nsteps,nsteps))
[i,j] = indices(Hamiltonian.shape)
Laplacian=(-2.0*diag(ones(nsteps))+diag(ones(nsteps-1),1)+diag(ones(nsteps-1),-1))/(float)(h**2)
#convert the internuclear distance to relative distance from R0
xgridr = (xgrid-R0)*sqrt(muau)
V=potential(xgridr,omega)
Hamiltonian[i==j]=V
Hamiltonian+=(-1.0/(2.0*muau))*Laplacian

energies, wavef = linalg.eigh(Hamiltonian)
print('Zero-point energy: {:3.2f} cm-1'.format(energies[0]/cm2Eh))
print('Fundamental vibration: {:3.2f} cm-1'.format((energies[1]-energies[0])/cm2Eh))

In [ ]:
#let's try a superposition state
#set the inital state
initstate = [1,2]
val = 1.0/sqrt(len(initstate))
#or set this manually if you do not want a 1:1 superposition
psi=wavef.flatten(order='F')
energy=zeros(nsteps*nsteps)
cvec = zeros((nsteps*nsteps),dtype=complex)
for i in range(nsteps):
    for j in range(nsteps):
        energy[i*nsteps+j] = energies[i]

for i in range(len(initstate)):
    cvec[(initstate[i]-1)*nsteps:initstate[i]*nsteps]=val
psi=psi*cvec
t0=0.0
t1fs = 100
dtfs = 1
tsteps = int(t1fs/dtfs)
t1 = t1fs/au2fs
dt = dtfs/au2fs
def f(t, y, arg1):
    return [-1j*arg1*y] 
r = ode(f).set_integrator('zvode', method='adams')
r.set_initial_value(psi, t0).set_f_params(energy)
    
psitemp=zeros((tsteps,nsteps*nsteps),dtype=complex)
tgrid=zeros(tsteps)
i=0
while r.successful() and r.t < t1:
    tgrid[i]=r.t+dt
    psitemp[i]=r.integrate(r.t+dt)
    i+=1

psit = psitemp.reshape(tsteps,nsteps,nsteps)
#build one psi
psiout = zeros((tsteps,nsteps),dtype=complex)
for i in range(tsteps):
    for j in range(nsteps):
        psiout[i] += psit[i,j]

In [ ]:
#plot
fig, ax = plt.subplots(tsteps,figsize=(5,3*tsteps),sharex=True,sharey=True)
for i in range(tsteps):
    ax[i].plot(xgrid,real(psiout[i]),label='real part')
    ax[i].plot(xgrid,imag(psiout[i]),label='imaginary part')
    ax[i].plot(xgrid,abs(psiout[i])**2,label='density')
    ax[i].text(xmin,0.17,'t = {:2.1f}fs'.format(tgrid[i]*au2fs),fontsize=14)
    ax[i].set_ylim(bottom=-0.2,top=0.2)
ax[0].set_title(r'$\Psi_{ODE}$',fontsize=18)

plt.subplots_adjust(wspace = 0.0)
plt.subplots_adjust(hspace = 0.0)
plt.show()

In [ ]:
def check_norm(psi):
    n = len(psi)
    c = zeros((n),dtype=complex)
    for i in range(n):
        c[i]=sum(conj(psi[i])*psi[i])
    return c

def aucofu(psi):
    n = len(psi)
    c = zeros((n),dtype=complex)
    for i in range(n):
        c[i]=sum(conj(psi[0])*psi[i])
    return c

In [ ]:
tt = check_norm(psit)
tt2 = aucofu(psit)

#plt.plot(real(tt))
#plt.plot(imag(tt))
#plt.plot(abs(tt)**2)

plt.plot(real(tt2))
plt.plot(imag(tt2))
plt.plot(abs(tt2)**2)

plt.show()

In [ ]:
def spec(c,eshift,tmax):
    fw = fft.ifft(c)
    n = len(c)//2
    dw = 2*pi/tmax
    norm = 1/tmax
    omega = (arange(n+1))*dw - eshift
    return fw[:n+1],omega

In [ ]:
fw,omega = spec(tt2,0.0,t1)
w=omega/cm2Eh
plt.plot(w,real(fw))
plt.plot(w,imag(fw))
plt.plot(w,abs(fw)**2)
plt.xlim(left=500,right=5500)
#plt.vlines((energies[1]-energies[0])/cm2Eh,-0.6,0.6)
plt.vlines((energies[0])/cm2Eh,-0.6,0.6)
plt.vlines((energies[1])/cm2Eh,-0.6,0.6)
plt.show()

How can we get a better resolution between the points? The sampling theorem states that the sampling rate in time domain connects to the maximum resolved frequency as 
\begin{align}
f_{max} = \frac{1}{2dt} \\
\omega_{max} = 2\pi f = \frac{\pi}{dt}
\end{align}

In [ ]:
print(omega[-1])
print(pi/((tgrid[1]-tgrid[0])))

To obtain a better frequency resolution, we need to propagate for a longer time, as then the lower frequencies are better resolved:
\begin{align}
\Delta f = \frac{1} {t_{max}} \\
\Delta \omega = \frac{2 \pi} {t_{max}}
\end{align}

In [ ]:
print(2*pi/tgrid[-1])

So let's try that and see the difference!

In [ ]:
t0=0.0
t2fs = 500
dtfs = 1
tsteps = int(t2fs/dtfs)
t1 = t2fs/au2fs
dt = dtfs/au2fs
thresh = 1e-8

r.set_initial_value(psi, t0).set_f_params(energy)
    
psitemp=zeros((tsteps,nsteps*nsteps),dtype=complex)
tgrid=zeros(tsteps)
i=0

#have to correct for inaccuracies when transforming to au through the use of a numerical threshold
while r.successful() and (t1-r.t) > thresh:
    tgrid[i]=r.t+dt
    psitemp[i]=r.integrate(r.t+dt)
    i+=1

psit = psitemp.reshape(tsteps,nsteps,nsteps)
#build one psi
psiout = zeros((tsteps,nsteps),dtype=complex)
for i in range(tsteps):
    for j in range(nsteps):
        psiout[i] += psit[i,j]

In [ ]:
tt = check_norm(psit)
tt2 = aucofu(psit)

plt.plot(real(tt))
plt.plot(imag(tt))
plt.plot(abs(tt)**2)

plt.plot(real(tt2))
plt.plot(imag(tt2))
plt.plot(abs(tt2)**2)

plt.show()

In [ ]:
fw,omega = spec(tt2,0.0,t1)
w=omega/cm2Eh
plt.plot(w,real(fw))
plt.plot(w,imag(fw))
plt.plot(w,abs(fw)**2)
plt.xlim(left=-500,right=5500)
#plt.vlines((energies[1]-energies[0])/cm2Eh,-0.6,0.6)
plt.vlines((energies[0])/cm2Eh,-0.6,0.6)
plt.vlines((energies[1])/cm2Eh,-0.6,0.6)
plt.show()

You can try this out for different superposition states and propagation lengths!

# Filtering methods and improving the FFT

Assume you have a noisy signal like this:

In [ ]:
import pandas as pd
fname = 'data_spec.dat'
expect = pd.read_csv('{}'.format(fname), sep='\s+')
time = expect.time.values/au2fs
etot = expect.Htot.values

#because of () we need to use .loc[]
sigt1 = array(expect.loc[:,'Re(Acf)'] + 1j*expect.loc[:,'Im(Acf)'])
plt.plot(time,real(sigt1))
plt.show()
plt.plot(time,imag(sigt1))
plt.show()

In [ ]:
fw,omega = spec(sigt1,0.0,time[-1])
mf=22
fig, ax = plt.subplots(figsize=(15,10))
ax.plot(omega,abs(fw))
ax.set_xlim(left=0,right=0.9)
ax.set_ylim(bottom=0,top=0.06)
ax.tick_params(labelsize=mf)
ax.set_xlabel('Energy (E$_h$)', fontsize=mf)
ax.set_ylabel('Intensity', fontsize=mf)
plt.show()

Can we clean this up somehow? 

1. Multiply with a function so that the autocorrelation function is cut off at the endpoint (reduce Gibb's phenomenon: spurious oscillations at interval boundaries). For example: 
\begin{align}
f = \cos\left(\frac{\pi t}{2T} \right)
\end{align}
where $T$ is the final time of the propagation.
Or, if you want to ensure both end points are zero and the function reflects periodicity in time:
\begin{align}
f = \sin\left(\frac{\pi t}{T}\right)
 \end{align}
 
You can also use a combination of both:
\begin{align}
f = \sin\left(\frac{\pi t}{T}\right) \exp \left( -\frac{t-T}{2\sigma T} \right)^2
 \end{align}
2. Use (sliding) window functions to extract time profiles of spectra.


3. Use an FFT as filtering methods to reduce noise.

In [ ]:
#1. 
plt.plot(time,cos(pi*time/(2*time[-1])))
plt.show()
f2 = cos(pi*time/(2*time[-1]))*sigt1
plt.plot(time,real(f2))
plt.plot(time,imag(f2))
plt.show()
fw2,omega2 = spec(f2,0.0,time[-1])
mf=22
fig, ax = plt.subplots(figsize=(15,10))
ax.plot(omega,abs(fw))
ax.plot(omega2,abs(fw2))
ax.set_xlim(left=0,right=0.9)
ax.set_ylim(bottom=0,top=0.06)
ax.tick_params(labelsize=mf)
ax.set_xlabel('Energy (E$_h$)', fontsize=mf)
ax.set_ylabel('Intensity', fontsize=mf)
plt.show()

In [ ]:
#1. 
plt.plot(time,sin(pi*time/(time[-1])))
plt.show()
f2 = sin(pi*time/(time[-1]))*sigt1
plt.plot(time,real(f2))
plt.plot(time,imag(f2))
plt.show()
fw2,omega2 = spec(f2,0.0,time[-1])
mf=22
fig, ax = plt.subplots(figsize=(15,10))
ax.plot(omega,abs(fw))
ax.plot(omega2,abs(fw2))
ax.set_xlim(left=0,right=0.9)
ax.set_ylim(bottom=0,top=0.06)
ax.tick_params(labelsize=mf)
ax.set_xlabel('Energy (E$_h$)', fontsize=mf)
ax.set_ylabel('Intensity', fontsize=mf)
plt.show()

In [ ]:
#1. 
sigma=0.8
myw = sin(pi*time/(time[-1]))*exp(-0.5*(time-time[-1])/(sigma*time[-1]))**2
plt.plot(time,myw)
plt.show()
f2 = myw*sigt1
plt.plot(time,real(f2))
plt.plot(time,imag(f2))
plt.show()
fw2,omega2 = spec(f2,0.0,time[-1])
mf=22
fig, ax = plt.subplots(figsize=(15,10))
ax.plot(omega,abs(fw))
ax.plot(omega2,abs(fw2))
ax.set_xlim(left=0,right=0.9)
ax.set_ylim(bottom=0,top=0.06)
ax.tick_params(labelsize=mf)
ax.set_xlabel('Energy (E$_h$)', fontsize=mf)
ax.set_ylabel('Intensity', fontsize=mf)
plt.show()

In [ ]:
#2. 
def window(time):
    myw = sin(pi*time/(time[-1]))*exp(-0.5*(time-time[-1])/(sigma*time[-1]))**2
    return myw
#for example, split the propagation into ten windows (so each is 50 fs)
nowindow = 10
lenw = len(time)//nowindow
print(lenw,time[0],time[lenw]*au2fs)
myw = sin(pi*time[0:lenw]/(time[lenw-1]))*exp(-0.5*(time[0:lenw]-time[lenw-1])/(sigma*time[lenw-1]))**2
#plt.plot(myw)
#plt.show()
maskw = zeros((nowindow,len(time)))
myf = zeros((nowindow,len(time)),dtype=complex)
myfw = zeros((nowindow,len(time)//2+1),dtype=complex)
for i in range(nowindow):
    i0 = i*lenw
    i1 = (i+1)*lenw
    print(i0,i1)
    arr1 = zeros(i0)
    arr2 = zeros(len(time)-i1)
    if i0 == 0:
        maskw[i] = concatenate([myw,arr2])
    else:
        maskw[i] = concatenate([arr1,myw,arr2])
    plt.plot(time,maskw[i])
    myf[i]=sigt1*maskw[i]
    myfw[i],omega = spec(myf[i],0.0,time[-1])
    
plt.show()

In [ ]:
fig, ax = plt.subplots(nowindow,figsize=(8,3*nowindow))
for i in range(nowindow):
    ax[i].plot(omega,abs(myfw[i]))
    ax[i].set_xlim(left=0,right=0.9)
    ax[i].set_ylim(bottom=0,top=0.06)
    ax[i].tick_params(labelsize=mf)
    ax[i].set_xlabel('Energy (E$_h$)', fontsize=mf)
    ax[i].set_ylabel('Intensity', fontsize=mf)
plt.show()

In [ ]:
plt.imshow(abs(myfw)-abs(myfw[0]),extent=(0,1000,0,nowindow),aspect=100)
#plt.imshow(abs(myfw),extent=(0,1000,0,nowindow),aspect=100)
plt.show()

Generally, we see that the low-frequency features originate in later times.

In [ ]:
# 3.
fw = fft.ifft(sigt1)
omega = fft.fftfreq(sigt1.size, d=time[1]-time[0])
omega = omega*2*pi
plt.plot(omega)
plt.show()
high_freq_fft = fw.copy()
# set the maximum frequency to 0.1 Eh
max_freq = 0.1
high_freq_fft[abs(omega) > max_freq] = 0
filtered_sig = fft.fft(high_freq_fft)
plt.figure(figsize=(10, 8))
plt.plot(time*au2fs, sigt1, label='Original signal')
plt.plot(time*au2fs, filtered_sig, linewidth=2, label='Filtered signal')
plt.xlim(left=0,right=100)
plt.xlabel('Time [fs]')
plt.ylabel('Amplitude')
plt.legend(loc='best')
plt.show()

In [ ]:
fw,omega = spec(sigt1,0.0,time[-1])
fw_filter,omega_filter = spec(filtered_sig,0.0,time[-1])

fig, ax = plt.subplots(figsize=(15,10))
ax.plot(omega,abs(fw))
ax.plot(omega_filter,abs(fw_filter))

ax.set_xlim(left=0,right=0.9)
ax.set_ylim(bottom=0,top=0.06)
ax.tick_params(labelsize=mf)
ax.set_xlabel('Energy (E$_h$)', fontsize=mf)
ax.set_ylabel('Intensity', fontsize=mf)
plt.show()

This is quite a brute force approach and for more elaborate use see https://docs.scipy.org/doc/scipy/reference/signal.html.

# Generate a movie from python plots
You can generate movies with python like this:

1. Save plots for each time step.

2. Convert into movie using ffmpeg.

In [ ]:
import os

mf=22
xmin = 0
xmax = 5
ymin=-0.2
ymax=0.2
xpos=xmin
ypos=0.17
namex='R'
namey='Wavefunction and probability'
mydir='image'
outfile='movie'

def make_movie(mf,xmin,xmax,ymin,ymax,xpos,ypos,namex,namey,mydir,outfile):
    #for i in range(tsteps):
    for i in range(1):
        fig, ax = plt.subplots(figsize=(8,5))
        plt.plot(xgrid,real(psiout[i]),label='real part')
        plt.plot(xgrid,imag(psiout[i]),label='imaginary part')
        plt.plot(xgrid,abs(psiout[i])**2,label='density')
        plt.text(xpos,ypos,'t = {:2.1f}fs'.format(tgrid[i]*au2fs),fontsize=mf)
        ax.set_ylim(bottom=ymin,top=ymax)
        ax.set_title(r'$\Psi_{ODE}$',fontsize=mf)
        ax.xaxis.set_tick_params(labelsize=mf)
        ax.yaxis.set_tick_params(labelsize=mf)
        ax.set_xlabel('{}'.format(namex),fontsize=mf)
        ax.set_ylabel('{}'.format(namey),fontsize=mf)
        plt.tight_layout()
        plt.savefig('{}/im{}.png'.format(mydir,i), dpi=300,bbox_inches='tight')
        plt.close()
        #plt.show()
# you may have to adjust the ffmpeg command according to your operating system etc; this is for MacOS
# you may also try other programs that concatenate frames into a movie
# please look at the ffmpeg documentation for the meaning of the different options
    cur = os.getcwd()
    os.chdir(mydir)
    os.system('ffmpeg -y -r 8 -i im%d.png -s 1920x1080 -c:v libx264 -pix_fmt yuv420p {}.mp4'.format(outfile))
    os.chdir(cur)
    
make_movie(mf,xmin,xmax,ymin,ymax,xpos,ypos,namex,namey,mydir,outfile)

In [ ]:
cur = os.getcwd()
print(cur)

# Task 1

Generate the spectrum of the harmonic oscillator for a different superposition than the one shown above. Plot the raw spectrum, and the smoothened spectrum using a cosine or sine function as in (1.) above. Do this for two different final times of the propagation, 100 fs and 500 fs.
Upload your plot(s) to moodle.

# Optional

If you want, you can add the autocorrelation function and FT to your program as possible analysis routines.